In [161]:
# Output:
# 1. Tiles: Cropped images from EM data
#    Output image name format:
#    <ConnectionName>_(<X-coord>,<Y-coord>,<Z-coord>).png
#    OR
#    <ConnectionName>_(<X-coord>,<Y-coord>,<Z-coord>)_rot<angle>.png
#
#    Example: C7_(53.2345,123.7634,3).png
#    Example: C7_(53.2345,123.7634,3)_rot50.png


###############################################################

In [162]:
# IMPORT MODULES
import numpy as np
import cv2
import glob
import re
import sys

In [178]:
# INPUT

image_stack_dir = "../P7_em_export_mip3"
image_filetype = "png"


endpoints_filename = "IGL_coordinates.txt"

tile_image_dir = "IGL_tiles_shift_rotate"

include_section_at_contact2 = False

# Displacement factor (speficy at mip0, will be scaled for specified mip_level later in the code)
# If set to 0, then no displaced tiles will be created
d = 300

# Initial tile_size from which a smaller centered tile will be cropped after rotations.
# Should be even number, specify at mip0, will be scaled in the code
init_tile_size = 7200

# Final tile_size. A tile of this size will be cropped from the bigger tile after rotations.
# Should be even number, specify at mip0, will be scaled in the code
tile_size = 2400

mip_level = 3

# Rotation angles for augmentation
angles = [45,90]

In [179]:
# Calculate mip_factor
mip_factor = 2**mip_level
print("Using mip level:", mip_level, "   mip_factor:", mip_factor)

# Adjust tile_size according to the mip_level
init_tile_size = init_tile_size/mip_factor
tile_size = tile_size/mip_factor

print("Adjusted init_tile_size:", init_tile_size)
print("Adjusted tile_size:", tile_size)

Using mip level: 3    mip_factor: 8
Adjusted init_tile_size: 900.0
Adjusted tile_size: 300.0


In [180]:
def sorted_nicely(l): 
    """ Sort the given iterable in the way that humans expect."""
    """Taken from: http://stackoverflow.com/questions/2669059/how-to-sort-alpha-numeric-set-in-python"""
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)
###END FUNCTION sorted_nicely(l)

def rotation(img, angle):
    #angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

In [181]:
# Read full image stack
image_filenames = sorted_nicely(glob.glob(image_stack_dir+"/*."+image_filetype))
print("Read "+str(len(image_filenames))+" images from "+image_stack_dir+".")
print("First image: "+image_filenames[0]+"\nLast image:  "+image_filenames[-1])

Read 2514 images from ../P7_em_export_mip3.
First image: ../P7_em_export_mip3/img_s0000.png
Last image:  ../P7_em_export_mip3/img_s2513.png


In [182]:
# Read contact points from the endpoints file and perform tiling
tile_position_cnt = 0

endpoints_file = open(endpoints_filename,'r')
#endpoints_file.readline()# Read the header
for line in endpoints_file:
    if not line.startswith('#') and line.strip() != "":
        line = line.strip().split('\t')
        name = line[0]
        
        # Temporary section to restart tile extraction
        #region_num = int(name.replace('R',''))
        #if region_num not in [73]:
        #    continue
            
        contact1 = np.array([float(v) for v in line[1].replace("(","").replace(")","").split(",")])
        contact2 = np.array([float(v) for v in line[2].replace("(","").replace(")","").split(",")])

        print("Connection:", name, "From:", contact1, "To:", contact2)
        
        # Calculate points along the line between contact1 and contact2 at each intersection with the Z-slice.
        V = contact2-contact1 # Line (Vector) from contact1 to contact2
        L = np.linalg.norm(V) # Distance between contact1 and contact2
        V_unit = V/L # Unit vector along V
        
        Z = abs(V[2]) # Number of Z-slices
        l = L/Z # Distance between the two consecutive points along the line from contact1 to contact2
        
        for n in range(0, int(Z)+int(include_section_at_contact2)):
            tile_position_cnt += 1
            P = contact1 + n * l * V_unit # Point on the vector V at the intersection of nth Z-slice after contact1
            print("Generating tiles around point P:", P)

            # Calculate center points for tiles. There will be 8 additional tiles using displacements in
            # all 8 directions.So total 9 tiles.
            d_range = [-d,0,d]
            if d == 0:
                d_range = [0]
            for dx in d_range:
                for dy in d_range:
                    # Strategy for shifting.
                    # Original tile and four shifted tiles (in either X or Y, not in both X and Y)
                    if dx == 0 or dy == 0 or abs(dx) != abs(dy):
                        P_shifted = P + np.array([dx,dy,0])
                        P_shifted_scaled = P_shifted/np.array([mip_factor,mip_factor,1])
                        P_shifted = P_shifted.astype(int)
                        P_shifted_scaled = P_shifted_scaled.astype(int)

                        #print(name, "P_shifted:", P_shifted, "   P_shifted_scaled:", P_shifted_scaled)
                        
                        # Read full image from the stack and get tile

                        # Get half init_tile_size, assuming that init_tile_size is an even number
                        init_tile_size_half = int(init_tile_size/2)
                        #print(init_tile_size, init_tile_size_half)

                        # Get half tile_size
                        tile_size_half = int(tile_size/2)

                        # Read full image of corresponding Z-slice
                        img = cv2.imread(image_filenames[int(P_shifted_scaled[2])],0) # Flag 0 tells opencv to read as grayscale image
                        #print("Full image shape:", img.shape, np.sum(img))

                        # Slice-out the tile from the full image. Note the order of Y and X coordinates.
                        init_tile = img[P_shifted_scaled[1]-init_tile_size_half:P_shifted_scaled[1]+init_tile_size_half,
                                        P_shifted_scaled[0]-init_tile_size_half:P_shifted_scaled[0]+init_tile_size_half]
                        #print("Tile image shape:", tile.shape, np.sum(tile))

                        # Crop final tile from init_tile and write as image
                        center = [int(init_tile.shape[0]/2), int(init_tile.shape[1]/2)]
                        #print("Center:", center)
                        tile = init_tile[center[0]-tile_size_half:center[0]+tile_size_half,
                                         center[1]-tile_size_half:center[1]+tile_size_half]
                        tile_image_filename = tile_image_dir + "/" + name + "_(" + ",".join(P_shifted.astype(str)) + ").png"
                        #print("Tile:", name, tile_image_filename)
                        cv2.imwrite(tile_image_filename, tile)
                        #print()

                        # Rotate init_tile, crop final tile from rotated init_tile and write as image
                        for angle in angles:
                            tile_rot = rotation(init_tile, angle)
                            #print(tile_rot.shape)
                            center = [int(tile_rot.shape[0]/2), int(tile_rot.shape[1]/2)]
                            #print("Center:", center)
                            tile = tile_rot[center[0]-tile_size_half:center[0]+tile_size_half,
                                            center[1]-tile_size_half:center[1]+tile_size_half]

                            tile_image_filename = tile_image_dir + "/" + name + "_(" + ",".join(P_shifted.astype(str)) + ")_rot" + str(angle) + ".png"
                            #print("Tile:", name, tile_image_filename)
                            cv2.imwrite(tile_image_filename, tile)
                            #print()

        
        print()
    #break

endpoints_file.close()
print("DONE")
print("Total number of tile positions:", tile_position_cnt)
print("Number of images per tile position:", 5*(len(angles)+1))
print("Total number of images written:", tile_position_cnt*5*(len(angles)+1))

Connection: R1 From: [20288. 40032.     0.] To: [2.0288e+04 4.0032e+04 1.0000e+00]
Generating tiles around point P: [20288. 40032.     0.]

Connection: R2 From: [12992. 39264.     0.] To: [1.2992e+04 3.9264e+04 1.0000e+00]
Generating tiles around point P: [12992. 39264.     0.]

Connection: R3 From: [13632. 44320.     0.] To: [1.3632e+04 4.4320e+04 1.0000e+00]
Generating tiles around point P: [13632. 44320.     0.]

Connection: R4 From: [20352. 44960.     0.] To: [2.0352e+04 4.4960e+04 1.0000e+00]
Generating tiles around point P: [20352. 44960.     0.]

Connection: R5 From: [32416. 43264.     0.] To: [3.2416e+04 4.3264e+04 1.0000e+00]
Generating tiles around point P: [32416. 43264.     0.]

Connection: R6 From: [34016. 41888.     0.] To: [3.4016e+04 4.1888e+04 1.0000e+00]
Generating tiles around point P: [34016. 41888.     0.]

Connection: R7 From: [30976. 47232.   217.] To: [30976. 47232.   218.]
Generating tiles around point P: [30976. 47232.   217.]

Connection: R8 From: [22480. 444


Connection: R65 From: [32592. 41328.  1831.] To: [32592. 41328.  1832.]
Generating tiles around point P: [32592. 41328.  1831.]

Connection: R66 From: [37312. 37936.  1831.] To: [37312. 37936.  1832.]
Generating tiles around point P: [37312. 37936.  1831.]

Connection: R67 From: [38720. 37312.  1870.] To: [38720. 37312.  1871.]
Generating tiles around point P: [38720. 37312.  1870.]

Connection: R68 From: [40688. 32112.  1870.] To: [40688. 32112.  1871.]
Generating tiles around point P: [40688. 32112.  1870.]

Connection: R69 From: [41248. 29952.  1892.] To: [41248. 29952.  1893.]
Generating tiles around point P: [41248. 29952.  1892.]

Connection: R70 From: [33712. 32752.  1892.] To: [33712. 32752.  1893.]
Generating tiles around point P: [33712. 32752.  1892.]

Connection: R71 From: [28352. 30608.  1936.] To: [28352. 30608.  1937.]
Generating tiles around point P: [28352. 30608.  1936.]

Connection: R72 From: [26416. 29392.  1995.] To: [26416. 29392.  1996.]
Generating tiles around 